<a href="https://colab.research.google.com/github/alvarosstsantos/fiap/blob/main/modulo_1_analise_exploratoria_de_dados/animated_geoplot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/alvarosstsantos/fiap.git

In [ ]:
!pip install pycountry
!pip install mapclassify
!pip install geoplot

In [ ]:
import pandas as pd
import pycountry
import matplotlib.pyplot as plt
import geopandas
import time
from IPython import display

In [ ]:
df = pd.read_csv("/content/fiap/modulo_1_analise_exploratoria_de_dados/datasets/EXP_2000_2023_20230831.csv", sep=";", decimal=",", thousands=".")

In [ ]:
df_filtered = df[["Country", "Year", "Net Weight", "US$ FOB"]].groupby(["Year","Country"], as_index=False).agg({"Net Weight": "sum", "US$ FOB": "sum"})

In [ ]:
mismatched_countries = {
    "Netherlands Antilles": "ANT",
    "Russia": "RUS",
    "Cape Verde": "CPV",
    "Democratic Republic of the Congo":"COD",
    "São Tome and Principe":"STP",
    "Bahrein": "BHR",
    "Iran": "IRN",
    "Dominica Island": "DMA",
    "French Guyana": "GUF",
    "Saint Vicent and the Granadines": "VCT",
    "Swaziland": "SWZ",
    "Saint Christopher and Nevis": "KNA"
}

In [ ]:
def alpha3code(col):
    cc=[]
    for c in col:
        code = None
        for l in (
            lambda c: pycountry.countries.get(name=c).alpha_3,
            lambda c: pycountry.countries.get(common_name=c).alpha_3,
            lambda c: pycountry.countries.get(official_name=c).alpha_3,
            lambda c: mismatched_countries[c]
        ):
            try:
                code = l(c)
            except:
                continue
            cc.append(code)
            break
    return cc

In [ ]:
df_filtered["ISO 3166-1 Country Code"] = alpha3code(df_filtered["Country"])

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))[["iso_a3", "continent", "geometry"]]
world.columns = ["ISO 3166-1 Country Code", "Continent", "Geometry"]
world.set_geometry("Geometry", inplace=True)
merge = pd.merge(world, df_filtered, on="ISO 3166-1 Country Code")
merge_ordered = merge.sort_values(by="Year")

In [ ]:
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(10, 10))

for y in merge_ordered["Year"].unique():
    world.plot(ax=ax, color="dimgrey",)
    merge_ordered[merge_ordered["Year"] == y].plot(
        ax = ax,
        column="Net Weight",
        cmap="Reds",
        scheme="quantiles",
        legend=True,
        legend_kwds={'loc': 'lower left'})

    plt.title("Net Weight - " + str(y), fontsize=12)

    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])

    display.display(plt.gcf())
    display.clear_output(wait=True)

    time.sleep(1)